# Lattice Bethe-Salpeter Equation (BSE) on the Hubbard atom

First we setup a dispersion for the lattice.

This can be done using a tight-binding Hamiltonian, see the square lattice guide.

In [2]:
from triqs_tprf.analytic_hubbard_atom import analytic_hubbard_atom

p = analytic_hubbard_atom(
    beta = 1.234,
    U = 5.0,
    nw = 1,
    nwf = 248,
    nwf_gf = 2 * 248,
    )

In [3]:
from pytriqs.lattice import BrillouinZone, BravaisLattice
bl = BravaisLattice([(1,0,0), (0,1,0)])
bz = BrillouinZone(bl)    

from pytriqs.gf import MeshBrillouinZone
bzmesh = MeshBrillouinZone(bz, n_k=1) # only one k-point

from pytriqs.gf import Gf
e_k = Gf(mesh=bzmesh, target_shape=[1, 1])
e_k *= 0.

In [4]:
Sigma_iw = p.G_iw.copy()
G0_iw = p.G_iw.copy()

from pytriqs.gf import inverse, iOmega_n
G0_iw << inverse(iOmega_n + 0.5*U)
Sigma_iw << inverse(G0_iw) - inverse(p.G_iw)

Green Function G_dn with mesh Matsubara Freq Mesh of size 992, Domain: Matsubara domain with beta = 1.234, statistic = Fermion, positive_only : 0 and target_rank 2: 

In [5]:
from triqs_tprf.lattice import lattice_dyson_g_wk
g_wk = lattice_dyson_g_wk(mu=0.5*U, e_k=e_k, sigma_w=Sigma_iw)

from triqs_tprf.lattice import fourier_wk_to_wr
g_wr = fourier_wk_to_wr(g_wk)

In [7]:
# ------------------------------------------------------------------
# -- Non-interacting generalized lattice susceptibility

from triqs_tprf.lattice import chi0r_from_gr_PH
chi0_wr = chi0r_from_gr_PH(nw=nw, nnu=nwf, gr=g_wr) 

from triqs_tprf.lattice import chi0q_from_chi0r
chi0_wk = chi0q_from_chi0r(chi0_wr)

In [9]:
# ------------------------------------------------------------------
# -- Solve lattice BSE

from triqs_tprf.lattice import chiq_from_chi0q_and_gamma_PH
chi_wk = chiq_from_chi0q_and_gamma_PH(chi0_wk, p.gamma_m)

# ------------------------------------------------------------------
# -- Store results and static results

#num = np.squeeze(parm.chi_wk.data.real)
#ref = np.squeeze(parm.chi_m.data.real)

#diff = np.max(np.abs(num - ref))
#print 'diff =', diff

#parm.chi_w = chiq_sum_nu_q(parm.chi_wk) # static suscept

#return parm